# TorchFort example train_distributed - version 2

2025-06-25

In [ ]:
! cp  /scratch${HOME#/prj}/sync/torchfort_local/examples/fortran/simulation  /scratch${HOME#/prj}/simu-f90

In [9]:
! singularity exec /scratch${HOME#/prj}/containers/torchfort.sif bash -c "pwd"

/prj/ampemi/eduardo.miranda2/sync


In [21]:
TFS="singularity exec /scratch${HOME#/prj}/containers/torchfort.sif"

In [11]:
! {TFS} cat /etc/os-release

PRETTY_NAME="Ubuntu 22.04.5 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.5 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [7]:
! {TFS} pwd

/prj/ampemi/eduardo.miranda2/sync


In [12]:
! {TFS} ls /

NGC-DL-CONTAINER-LICENSE    environment  lib64	 prj   singularity  var
bin			    etc		 libx32  proc  srv	    yaml-cpp
boot			    home	 media	 root  sys
cuda-keyring_1.1-1_all.deb  lib		 mnt	 run   tmp
dev			    lib32	 opt	 sbin  usr


In [23]:
! {TFS} ls /opt/torchfort/bin/examples/fortran/simulation

config_fcn_torchscript.yaml  generate_fcn_model.py  train_distributed
config_mlp_native.yaml	     train		    visualize.py


In [27]:
%%bash
singularity exec "/scratch${HOME#/prj}/containers/torchfort.sif" \
    cp /opt/torchfort/bin/examples/fortran/simulation/ .

In [28]:
! mv ./train_distributed  /scratch${HOME#/prj}/simu-f90/

Notes

- The submission .srm file and the execution file must be in the same "workdir" directory in /scratch/, or at least within the .srm there must be a "cd workdir" before the executable, otherwise the TorchFort example file (train_distributed.f90) will not find the configuration file. 

In [92]:
a = ! echo $HOME
WDR = a[0].replace('/prj/','/scratch/') + "/simu-f90"
%cd {WDR}

/scratch/ampemi/eduardo.miranda2/simu-f90


In [93]:
%%writefile train_distrib.srm
#!/bin/bash
#SBATCH --job-name=train_distrib      # SLURM_JOB_NAME
#SBATCH --partition=sequana_gpu_dev   # SLURM_JOB_PARTITION
#SBATCH --time=00:05:00               # Limit execution time
#SBATCH --nodes=1                     # SLURM_JOB_NUM_NODES
#SBATCH --ntasks-per-node=2           # SLURM_NTASKS_PER_NODE
#SBATCH --gpus=2                      # SLURM_GPUS
#SBATCH --gpus-per-node=2             # SLURM_GPUS_PER_NODE
#SBATCH --distribution=block:cyclic   # SLURM_DISTRIBUTION
    # Not used:
    # Not compatible with the --cpus-per-gpu option:
    # #SBATCH --cpus-per-task=1             # SLURM_CPUS_PER_TASK
    # #SBATCH --cpus-per-gpu=1              # SLURM_CPUS_PER_GPU
## SLURM output environment variables: 
# https://slurm.schedmd.com/sbatch.html#SECTION_OUTPUT-ENVIRONMENT-VARIABLES
export WDIR=
echo '======================================='
echo '- Job ID:' $SLURM_JOB_ID
echo '- Nº of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- Nº of tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- Nº of tasks:' $SLURM_NTASKS
echo '- Nº of cpus per task:' $SLURM_CPUS_PER_TASK
echo '- Partition:' $SLURM_JOB_PARTITION
echo -n '- Nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST
echo '- Submit directory:' $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR

module load openmpi/gnu/4.1.4+gcc-12.4+cuda-11.6_sequana
export SLURM_MPI_TYPE=pmi2
export GMX_ENABLE_DIRECT_GPU_COMM=1
export IMG=/scratch${HOME#/prj}/containers/torchfort.sif
export SGL="singularity exec --nv --bind /scratch:/scratch $IMG"
export WDR=/scratch${HOME#/prj}/simu-f90
cd $WDR
echo '- Work directory:  ' $WDR
echo -n '<1. starting>       ' && date
echo '-- output -----------------------------'
srun -n $SLURM_NTASKS $SGL bash -c "mpirun --allow-run-as-root $WDR/train_distributed"
echo '-- end --------------------------------'
echo -n '<2. end>            ' && date

Overwriting train_distrib.srm


In [94]:
N="train_distrib"
import time
a = !sbatch {N}.srm
time.sleep(1)
!squeue --me --name={N}
while True:
    b = !squeue --me --name={N}
    if len(b) < 2 : break
    print(".",end="")
    time.sleep(30)
print()
a = a[0].replace('Submitted batch job ','')
b = %pwd
b = b.replace('/prj/','/scratch/')
%cat {b}/slurm-{a}.out

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          11358855 sequana_g train_di eduardo.  R       0:00      1 sdumont8046
...........
- Job ID: 11358855
- Nº of nodes in the job: 1
- Nº of tasks per node: 2
- Nº of tasks: 2
- Nº of cpus per task:
- Partition: sequana_gpu_dev
- Nodes allocated to the job: sdumont8046
- Submit directory: /scratch/ampemi/eduardo.miranda2/simu-f90
- Work directory:   /scratch/ampemi/eduardo.miranda2/simu-f90
<1. starting>       qua jun 25 16:17:57 -03 2025
-- output -----------------------------
 Run settings:
 	configfile: config_mlp_native.yaml
 	simulation_device: gpu
 	train_device: gpu
 	checkpoint_dir:NONE
 	output_model_name: model.pt
 	output_checkpoint_dir: checkpoint
 	ntrain_steps:        100000
 	nval_steps:          1000
 	val_write_freq:            10
 
 Run settings:
 	configfile: config_mlp_native.yaml
 	simulation_device: gpu
 	train_device: gpu
 	checkpoint_dir:NONE
 	output_model_name: model.pt
